In [7]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import pearsonr

# Load the datasets into Dataframes
df = pd.read_csv('psychiatrists-working-in-the-mental-health-sector.csv')
source1_df = pd.read_csv('death-rates-from-mental-and-substance-disorders-by-age-who.csv')

# Merge the DataFrames based on both 'Country' and 'Year' columns
merged_df = pd.merge(df, source1_df, on=['Entity', 'Year'], how='inner')

# Save the merged data to modified_data.csv
df_final = merged_df[merged_df["Year"] == 2016]
df_final.to_csv('merged_data.csv', index=False)

In [8]:
# Hypothesis Testing
# Observed correlation
observed_correlation, _ = pearsonr(df_final["Indicator:Psychiatrists working in mental health sector (per 100 000 population)"], df_16["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"])

# Permutation test
def permutation_test(data, num_permutations=1000):
    permuted_correlations = []
    for _ in range(num_permutations):
        permuted_data = data.copy()
        permuted_data["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"] = np.random.permutation(permuted_data["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"])
        permuted_correlation, _ = pearsonr(permuted_data["Indicator:Psychiatrists working in mental health sector (per 100 000 population)"], permuted_data["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"])
        permuted_correlations.append(permuted_correlation)
    return np.array(permuted_correlations)

permuted_correlations = permutation_test(df_16)

# p-value calculation
p_value = np.sum(np.abs(permuted_correlations) >= np.abs(observed_correlation)) / len(permuted_correlations)

print(f"Observed Correlation: {observed_correlation}")
print(f"Permutation Test p-value: {p_value}")
# H0 is null hypothesis that there is no difference
# H0 is null hypothesis that there is no difference
# Hypothesis testing
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference.")

Observed Correlation: 0.4440224728902288
Permutation Test p-value: 0.001
Reject the null hypothesis: There is a significant difference.


In [9]:
# Perform bootstrapping
def bootstrap_correlation(data, num_samples=1000):
    correlations = []
    for _ in range(num_samples):
        sample = data.sample(frac=1, replace=True)
        correlation = sample["Indicator:Psychiatrists working in mental health sector (per 100 000 population)"].corr(sample["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"])
        correlations.append(correlation)
    return np.array(correlations)

np.random.seed(0)  # For reproducibility
bootstrap_correlations = bootstrap_correlation(df_final)

# Calculate the mean correlation and confidence intervals
mean_correlation = bootstrap_correlations.mean()
conf_interval = np.percentile(bootstrap_correlations, [2.5, 97.5])

print(f"Bootstrap Mean Correlation: {mean_correlation}")
print(f"95% Confidence Interval: {conf_interval}")
# Check if zero is within the confidence interval
significant_difference = not (conf_interval[0] <= 0 <= conf_interval[1])

print(f"Significant difference: {significant_difference }")



Bootstrap Mean Correlation: 0.4678628296803498
95% Confidence Interval: [0.33319653 0.6328219 ]
Significant difference: True
